# Program pembuat model kelas 19 pake GoogleColab

sumber : https://youtu.be/-bN-R1D1Pe4

In [1]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, LSTM
from keras.callbacks import ModelCheckpoint

In [2]:
# menghubungkan drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# mengimport file python/modul python yang ada di drive
import sys
sys.path.insert(0,'/content/drive/MyDrive/iky/fungsi')
import datanya

In [4]:
timestep = 80

# ~~~~~ file yang berisi ticker jadikan sebuah list
with open('drive/MyDrive/iky/data/ticker_issi.txt') as file:
    tick = file.readlines()
    ticker = [line.rstrip() for line in tick]# ini adalah list yang berisi ticker

In [19]:
ticker = ['APEX']#['YPAS'] # 

In [ ]:
for tic in ticker:
    # df = pd.read_csv(f'drive/MyDrive/iky/data/bsaham/{tic}.csv')
    df = pd.read_csv(f'drive/MyDrive/campuran/data/saham/{tic}.csv')
    x_train, y_train = datanya.data_latih12(df, timestep)
    model = Sequential()
    model.add(LSTM(units=80, return_sequences=True, input_shape=(x_train.shape[1], 1)))
    model.add(Dropout(0.2))
    model.add(LSTM(units=80, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=80))
    model.add(Dropout(0.2))
    model.add(Dense(units=40))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    # filepath = f"drive/MyDrive/iky/kelas_9/model/{tic}_model.h5"
    # checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True, mode='min')
    # callbacks_list = [checkpoint]
    # model.fit(X_train, y_train, batch_size=cur_batch_size, epochs=cur_epochs, callbacks=callbacks_list,
    #           shuffle=True)
    model.fit(x_train, y_train, epochs=2, batch_size=1)

In [7]:
# ~~~~~~~~~~~~~~~~ fungsi untuk menggunakan model kelas 1 untuk memprediksi

import pandas as pd
from keras.models import load_model
from datetime import date

## fungsi untuk menghitung kenaikan harga
def kenaikan_persen(nilai_awal, nilai_terakhir):
    persen = float(((nilai_terakhir-nilai_awal)/nilai_awal)*100)
    if persen >= 0:
        akan = [1,'Naik']
    elif persen <= 0:
        akan = [0,'Turun']
    return persen, akan

def pake_k20(list_ticker, timestep=80):
    hasil_model = pd.DataFrame(columns=['Saham','Perubahan nilai','Keadaan','0/1','Perubahan(%)'])
    for tic in list_ticker:
        makanannya = pd.read_csv(f'drive/MyDrive/campuran/data/saham/{tic}.csv')
        # model = load_model(f"kelas_20/model/{tic}_model.h5")
        hasil_pred = datanya.predik_5h(makanannya, timestep, model)
        kenaikan = hasil_pred[4][0] - hasil_pred[0][0]
        persentase, akan = kenaikan_persen(hasil_pred[0][0], hasil_pred[4][0])
        hasil_model = pd.concat([hasil_model, pd.DataFrame.from_records([{
            'Saham': tic, 'Perubahan nilai' : kenaikan, 'Keadaan': akan[1],
            '0/1':akan[0],'Perubahan(%)' : persentase}])])
    # mengurutkan sesuai kolom Perubahan(%), dari nilai terbesar ke terkecil
    hasil_model.sort_values(by='Perubahan(%)',ascending=False, inplace=True) 
    hasil_model.reset_index(inplace=True, drop=True) # mereset index
    return hasil_model

In [ ]:
hasil_k20 = pake_k20(ticker)
hasil_k20